## Comparation gnome sort and comb short

In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import random
from typing import List, Tuple, Optional

# Load dataset
file_path = r"C:\Users\akip\Desktop\comparation-search-or-sort-algoritm\dummy_transactions_10000.csv"
df = pd.read_csv(file_path)

print("Dataset Info:")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"Data Types:\n{df.dtypes}")
print("\nFirst 5 rows:")
print(df.head())

Dataset Info:
Shape: (10000, 4)
Columns: ['transaction_id', 'customer_name', 'amount', 'date']
Data Types:
transaction_id     object
customer_name      object
amount            float64
date               object
dtype: object

First 5 rows:
  transaction_id  customer_name   amount                        date
0       TX100000    Customer_49  1878.96  2025-10-16 17:00:33.160554
1       TX100001  Customer_1105  4754.06  2025-10-19 17:00:33.160554
2       TX100002   Customer_342  3662.65  2025-11-15 17:00:33.160554
3       TX100003   Customer_232  2997.31  2025-11-19 17:00:33.160554
4       TX100004   Customer_650   788.53  2025-10-24 17:00:33.160554


In [ ]:
# Implementasi Gnome Sort Algorithm
def gnome_sort(arr: List, key_attr: str = None) -> Tuple[List, int, int]:
    """
    Implementasi Gnome Sort Algorithm
    
    Args:
        arr: List of elements to sort
        key_attr: Attribute name if sorting list of objects
    
    Returns:
        Tuple: (sorted_array, comparisons, swaps)
    """
    if len(arr) <= 1:
        return arr.copy(), 0, 0
    
    arr_copy = arr.copy()
    n = len(arr_copy)
    index = 0
    comparisons = 0
    swaps = 0
    
    while index < n:
        comparisons += 1
        
        if index == 0:
            index += 1
        else:
            # Bandingkan elemen current dan previous
            current_val = getattr(arr_copy[index], key_attr) if key_attr else arr_copy[index]
            prev_val = getattr(arr_copy[index-1], key_attr) if key_attr else arr_copy[index-1]
            
            comparisons += 1
            if current_val >= prev_val:
                index += 1
            else:
                # Swap elemen
                arr_copy[index], arr_copy[index-1] = arr_copy[index-1], arr_copy[index]
                swaps += 1
                index -= 1
    
    return arr_copy, comparisons, swaps

In [ ]:
# Implementasi Comb Sort Algorithm
def comb_sort(arr: List, key_attr: str = None) -> Tuple[List, int, int]:
    """
    Implementasi Comb Sort Algorithm
    
    Args:
        arr: List of elements to sort
        key_attr: Attribute name if sorting list of objects
    
    Returns:
        Tuple: (sorted_array, comparisons, swaps)
    """
    if len(arr) <= 1:
        return arr.copy(), 0, 0
    
    arr_copy = arr.copy()
    n = len(arr_copy)
    gap = n
    shrink_factor = 1.3
    sorted_flag = False
    comparisons = 0
    swaps = 0
    
    while not sorted_flag:
        # Update gap
        gap = int(gap / shrink_factor)
        if gap <= 1:
            gap = 1
            sorted_flag = True
        
        i = 0
        while i + gap < n:
            comparisons += 1
            
            # Bandingkan elemen dengan jarak gap
            current_val = getattr(arr_copy[i], key_attr) if key_attr else arr_copy[i]
            gap_val = getattr(arr_copy[i + gap], key_attr) if key_attr else arr_copy[i + gap]
            
            if current_val > gap_val:
                # Swap elemen
                arr_copy[i], arr_copy[i + gap] = arr_copy[i + gap], arr_copy[i]
                swaps += 1
                sorted_flag = False
            
            i += 1
    
    return arr_copy, comparisons, swaps

In [ ]:
# Persiapan Data untuk Testing
# Buat array untuk testing
class Transaction:
    def __init__(self, transaction_id, customer_name, amount, date):
        self.transaction_id = transaction_id
        self.customer_name = customer_name
        self.amount = float(amount)
        self.date = date
    
    def __repr__(self):
        return f"Transaction({self.transaction_id}, {self.customer_name}, {self.amount})"

# Konversi dataframe ke list of Transaction objects
transactions = []
for _, row in df.iterrows():
    transactions.append(Transaction(
        row['transaction_id'],
        row['customer_name'],
        row['amount'],
        row['date']
    ))

print(f"Total transactions: {len(transactions)}")
print(f"Sample transaction amounts: {[t.amount for t in transactions[:5]]}")

# Buat berbagai tipe data untuk testing
def prepare_test_cases(base_data, sizes):
    """Menyiapkan berbagai test case dengan ukuran berbeda"""
    test_cases = {}
    
    for size in sizes:
        # Sample data
        sample_data = random.sample(base_data, size)
        
        # Best case: data sudah terurut
        best_case = sorted(sample_data, key=lambda x: x.amount)
        
        # Average case: data acak
        average_case = sample_data.copy()
        
        # Worst case: data terurut terbalik
        worst_case = sorted(sample_data, key=lambda x: x.amount, reverse=True)
        
        test_cases[size] = {
            'best': best_case,
            'average': average_case,
            'worst': worst_case
        }
    
    return test_cases

# Siapkan test cases
test_sizes = [100, 500, 1000]
test_cases = prepare_test_cases(transactions, test_sizes)

print(f"Test cases prepared for sizes: {test_sizes}")

In [ ]:
#  Fungsi Testing dan Pengukuran Performa
def test_sorting_algorithms(test_cases_dict: dict) -> pd.DataFrame:
    """
    Test kedua algoritma sorting dengan berbagai test case
    """
    results = []
    
    for size, cases in test_cases_dict.items():
        for case_type, data in cases.items():
            print(f"Testing size {size}, {case_type} case...")
            
            # Test Gnome Sort
            start_time = time.time()
            gnome_sorted, gnome_comparisons, gnome_swaps = gnome_sort(data, 'amount')
            gnome_time = time.time() - start_time
            
            # Test Comb Sort
            start_time = time.time()
            comb_sorted, comb_comparisons, comb_swaps = comb_sort(data, 'amount')
            comb_time = time.time() - start_time
            
            # Verifikasi hasil sorting
            gnome_verified = verify_sorted(gnome_sorted, 'amount')
            comb_verified = verify_sorted(comb_sorted, 'amount')
            
            results.append({
                'data_size': size,
                'case_type': case_type,
                'gnome_time': gnome_time,
                'gnome_comparisons': gnome_comparisons,
                'gnome_swaps': gnome_swaps,
                'gnome_verified': gnome_verified,
                'comb_time': comb_time,
                'comb_comparisons': comb_comparisons,
                'comb_swaps': comb_swaps,
                'comb_verified': comb_verified
            })
    
    return pd.DataFrame(results)

def verify_sorted(arr: List, key_attr: str = None) -> bool:
    """Verifikasi apakah array sudah terurut"""
    for i in range(1, len(arr)):
        current = getattr(arr[i], key_attr) if key_attr else arr[i]
        prev = getattr(arr[i-1], key_attr) if key_attr else arr[i-1]
        if current < prev:
            return False
    return True

# Jalankan testing
print("Starting comprehensive testing...")
results_df = test_sorting_algorithms(test_cases)
print("Testing completed!")
print(results_df)

In [ ]:
# Analisis Kompleksitas Waktu dan Ruang
def analyze_theoretical_complexity():
    """
    Analisis kompleksitas waktu dan ruang untuk kedua algoritma
    """
    print("=" * 70)
    print("ANALISIS KOMPLEKSITAS TEORITIS")
    print("=" * 70)
    
    print("\n📊 GNOME SORT")
    print("Kompleksitas Waktu:")
    print("- Best Case    : O(n)     - Data sudah terurut")
    print("- Average Case : O(n²)    - Data acak")
    print("- Worst Case   : O(n²)    - Data terurut terbalik")
    print("\nKompleksitas Ruang: O(1) - In-place sorting")
    print("\nKarakteristik:")
    print("- Mirip dengan Insertion Sort tetapi dengan logika berbeda")
    print("- Tidak memerlukan nested loop eksplisit")
    print("- Efisien untuk data yang hampir terurut")
    
    print("\n📊 COMB SORT")
    print("Kompleksitas Waktu:")
    print("- Best Case    : O(n log n) - Data sudah terurut")
    print("- Average Case : O(n²/2^p)  - p adalah jumlah increment")
    print("- Worst Case   : O(n²)      - Data terurut terbalik")
    print("\nKompleksitas Ruang: O(1) - In-place sorting")
    print("\nKarakteristik:")
    print("- Improvement dari Bubble Sort")
    print("- Menggunakan shrink factor (biasanya 1.3)")
    print("- Efisien untuk data berukuran besar")
    
    print("\n🎯 PERBANDINGAN KOMPLEKSITAS")
    comparison_data = {
        'Algorithm': ['Gnome Sort', 'Comb Sort'],
        'Best Case': ['O(n)', 'O(n log n)'],
        'Average Case': ['O(n²)', 'O(n²/2^p)'],
        'Worst Case': ['O(n²)', 'O(n²)'],
        'Space Complexity': ['O(1)', 'O(1)'],
        'Stable': ['Yes', 'No']
    }
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))

analyze_theoretical_complexity()

In [ ]:
# Visualisasi Perbandingan Performa
def visualize_results(results_df: pd.DataFrame):
    """
    Visualisasi perbandingan performa kedua algoritma
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Case types dan colors
    case_types = ['best', 'average', 'worst']
    colors = ['green', 'orange', 'red']
    
    # Plot 1: Waktu eksekusi per case type
    for i, case_type in enumerate(case_types):
        case_data = results_df[results_df['case_type'] == case_type]
        
        axes[0, 0].plot(case_data['data_size'], case_data['gnome_time'], 
                       'o-', label=f'Gnome {case_type}', color=colors[i], linewidth=2)
        axes[0, 0].plot(case_data['data_size'], case_data['comb_time'], 
                       's--', label=f'Comb {case_type}', color=colors[i], linewidth=2)
    
    axes[0, 0].set_xlabel('Data Size')
    axes[0, 0].set_ylabel('Time (seconds)')
    axes[0, 0].set_title('Execution Time Comparison')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Perbandingan jumlah komparasi
    for i, case_type in enumerate(case_types):
        case_data = results_df[results_df['case_type'] == case_type]
        
        axes[0, 1].plot(case_data['data_size'], case_data['gnome_comparisons'], 
                       'o-', label=f'Gnome {case_type}', color=colors[i], linewidth=2)
        axes[0, 1].plot(case_data['data_size'], case_data['comb_comparisons'], 
                       's--', label=f'Comb {case_type}', color=colors[i], linewidth=2)
    
    axes[0, 1].set_xlabel('Data Size')
    axes[0, 1].set_ylabel('Number of Comparisons')
    axes[0, 1].set_title('Comparisons Count')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Perbandingan jumlah swaps
    for i, case_type in enumerate(case_types):
        case_data = results_df[results_df['case_type'] == case_type]
        
        axes[0, 2].plot(case_data['data_size'], case_data['gnome_swaps'], 
                       'o-', label=f'Gnome {case_type}', color=colors[i], linewidth=2)
        axes[0, 2].plot(case_data['data_size'], case_data['comb_swaps'], 
                       's--', label=f'Comb {case_type}', color=colors[i], linewidth=2)
    
    axes[0, 2].set_xlabel('Data Size')
    axes[0, 2].set_ylabel('Number of Swaps')
    axes[0, 2].set_title('Swaps Count')
    axes[0, 2].legend()
    axes[0, 2].grid(True, alpha=0.3)
    
    # Plot 4: Speedup ratio (Gnome/Comb)
    for i, case_type in enumerate(case_types):
        case_data = results_df[results_df['case_type'] == case_type]
        speedup_ratio = case_data['gnome_time'] / case_data['comb_time']
        
        axes[1, 0].plot(case_data['data_size'], speedup_ratio, 
                       'o-', label=case_type, color=colors[i], linewidth=2)
    
    axes[1, 0].axhline(y=1, color='black', linestyle='--', label='Break Even')
    axes[1, 0].set_xlabel('Data Size')
    axes[1, 0].set_ylabel('Gnome Time / Comb Time')
    axes[1, 0].set_title('Speedup Ratio (Higher = Comb Better)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 5: Memory efficiency (diasumsikan O(1) untuk kedua algoritma)
    sizes = results_df['data_size'].unique()
    memory_usage = [1] * len(sizes)  # Konstan untuk kedua algoritma
    
    axes[1, 1].plot(sizes, memory_usage, 'o-', label='Both Algorithms', linewidth=2)
    axes[1, 1].set_xlabel('Data Size')
    axes[1, 1].set_ylabel('Relative Memory Usage')
    axes[1, 1].set_title('Space Complexity (O(1) for both)')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    # Plot 6: Performance summary
    summary_data = results_df.groupby('case_type')[['gnome_time', 'comb_time']].mean()
    x = np.arange(len(summary_data))
    width = 0.35
    
    axes[1, 2].bar(x - width/2, summary_data['gnome_time'], width, label='Gnome Sort', alpha=0.7)
    axes[1, 2].bar(x + width/2, summary_data['comb_time'], width, label='Comb Sort', alpha=0.7)
    axes[1, 2].set_xlabel('Case Type')
    axes[1, 2].set_ylabel('Average Time (seconds)')
    axes[1, 2].set_title('Average Performance by Case Type')
    axes[1, 2].set_xticks(x)
    axes[1, 2].set_xticklabels(summary_data.index)
    axes[1, 2].legend()
    
    plt.tight_layout()
    plt.show()

visualize_results(results_df)

In [ ]:
# Analisis Detail Berdasarkan Tipe Kasus
def detailed_case_analysis(results_df: pd.DataFrame):
    """
    Analisis detail untuk setiap tipe kasus
    """
    print("=" * 70)
    print("ANALISIS DETAIL BERDASARKAN TIPE KASUS")
    print("=" * 70)
    
    for case_type in ['best', 'average', 'worst']:
        case_data = results_df[results_df['case_type'] == case_type]
        
        print(f"\n📈 {case_type.upper()} CASE ANALYSIS:")
        print(f"{'-' * 40}")
        
        for size in case_data['data_size'].unique():
            size_data = case_data[case_data['data_size'] == size].iloc[0]
            
            print(f"\nData Size: {size}")
            print(f"Gnome Sort  - Time: {size_data['gnome_time']:.6f}s, Comparisons: {size_data['gnome_comparisons']:,}")
            print(f"Comb Sort   - Time: {size_data['comb_time']:.6f}s, Comparisons: {size_data['comb_comparisons']:,}")
            
            speedup = size_data['gnome_time'] / size_data['comb_time']
            print(f"Speedup: {speedup:.2f}x ({'Comb' if speedup > 1 else 'Gnome'} lebih cepat)")

detailed_case_analysis(results_df)

In [ ]:
# Kesimpulan dan Rekomendasi
def print_conclusions_and_recommendations(results_df: pd.DataFrame):
    """
    Menampilkan kesimpulan dan rekomendasi berdasarkan analisis
    """
    print("=" * 70)
    print("KESIMPULAN DAN REKOMENDASI")
    print("=" * 70)
    
    # Hitung statistik overall
    avg_gnome_time = results_df['gnome_time'].mean()
    avg_comb_time = results_df['comb_time'].mean()
    overall_speedup = avg_gnome_time / avg_comb_time
    
    print(f"\n📊 HASIL PENGUJIAN EMPIRIS:")
    print(f"Rata-rata waktu Gnome Sort: {avg_gnome_time:.6f} detik")
    print(f"Rata-rata waktu Comb Sort:  {avg_comb_time:.6f} detik")
    print(f"Speedup overall: {overall_speedup:.2f}x")
    
    print("\n✅ GNOME SORT:")
    print("Kelebihan:")
    print("   - Implementasi sangat sederhana")
    print("   - Efisien untuk data kecil atau hampir terurut")
    print("   - Stable sort (mempertahankan urutan relatif)")
    print("Kekurangan:")
    print("   - O(n²) untuk kasus rata-rata dan terburuk")
    print("   - Tidak efisien untuk data besar")
    
    print("\n✅ COMB SORT:")
    print("Kelebihan:")
    print("   - Lebih efisien daripada Bubble Sort dan Gnome Sort")
    print("   - Performa mendekati O(n log n) untuk banyak kasus")
    print("   - Efisien untuk data berukuran besar")
    print("Kekurangan:")
    print("   - Tidak stable")
    print("   - Implementasi lebih kompleks daripada Gnome Sort")
    
    print("\n🎯 REKOMENDASI PENGGUNAAN:")
    print("Gunakan GNOME SORT ketika:")
    print("   - Data berukuran kecil (n < 100)")
    print("   - Data sudah hampir terurut")
    print("   - Implementasi sederhana diutamakan")
    print("   - Membutuhkan stable sort")
    
    print("\nGunakan COMB SORT ketika:")
    print("   - Data berukuran menengah hingga besar")
    print("   - Performa lebih penting daripada simplicity")
    print("   - Data memiliki distribusi acak")
    print("   - Tidak memerlukan stable sort")
    
    print("\n⚠️  CATATAN PENTING:")
    print("   - Kedua algoritma memiliki kompleksitas ruang O(1)")
    print("   - Comb Sort umumnya lebih cepat untuk data besar")
    print("   - Gnome Sort lebih mudah diimplementasi dan dipahami")

print_conclusions_and_recommendations(results_df)

In [ ]:
# Save Hasil Analisis
# Simpan hasil analisis ke file
def save_analysis_results(results_df: pd.DataFrame):
    """Menyimpan hasil analisis ke file CSV"""
    
    # Summary statistics
    summary_stats = results_df.groupby('case_type').agg({
        'gnome_time': ['mean', 'std'],
        'comb_time': ['mean', 'std'],
        'gnome_comparisons': 'mean',
        'comb_comparisons': 'mean'
    }).round(6)
    
    print("SUMMARY STATISTICS:")
    print(summary_stats)
    
    # Save detailed results
    results_df.to_csv('sorting_algorithms_comparison.csv', index=False)
    
    # Save complexity analysis
    complexity_data = {
        'Algorithm': ['Gnome Sort', 'Comb Sort'],
        'Best_Case_Time': ['O(n)', 'O(n log n)'],
        'Average_Case_Time': ['O(n²)', 'O(n²/2^p)'],
        'Worst_Case_Time': ['O(n²)', 'O(n²)'],
        'Space_Complexity': ['O(1)', 'O(1)'],
        'Stable': ['Yes', 'No'],
        'In_Place': ['Yes', 'Yes']
    }
    
    complexity_df = pd.DataFrame(complexity_data)
    complexity_df.to_csv('sorting_algorithms_complexity.csv', index=False)
    
    print("\n✅ Hasil analisis disimpan ke:")
    print("   - sorting_algorithms_comparison.csv")
    print("   - sorting_algorithms_complexity.csv")

save_analysis_results(results_df)

print("\n" + "="*70)
print("ANALISIS SELESAI!")
print("="*70)
